In [59]:
!python -V

Python 3.10.18


In [60]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [62]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1759671388532, experiment_id='2', last_update_time=1759671388532, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [ ]:
# cd /workspaces/MLOps-zoomcamp/02-experiment-tracking
# mlflow ui --backend-store-uri sqlite:///mlflow.db

In [64]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [65]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [66]:
len(df_train), len(df_val)

(73908, 61921)

In [67]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [68]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [69]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [70]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715208009878

In [71]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [72]:
with mlflow.start_run():
    mlflow.set_tag("developer", "kai")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [73]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.metrics import mean_squared_error

In [74]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [75]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        for key, value in params.items():
            mlflow.log_param(key, value)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,"validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse=root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}


In [76]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed' : 42,
# }

# best_result = fmin(
#     fn = objective,
#     space = search_space,
#     algo = tpe.suggest,
#     max_evals = 50,
#     trials = Trials()
# )

In [77]:
best_params = {
    'learning_rate':0.6545973055915408,
    'max_depth':33,
    'min_child_weight':3.3858551804076,
    'objective':'reg:linear',
    'reg_alpha':0.29589767192164507,
    'reg_lambda': 0.0579918889780704,
    'seed':42
}

mlflow.log_params(best_params)
 
booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid,"validation")],
            early_stopping_rounds=50
        )

y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)
mlflow.log_metric("rmse", rmse)

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/conda/envs/exp-tracking-env/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [10:26:05] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:7.68673
[1]	validation-rmse:6.80080
[2]	validation-rmse:6.61920
[3]	validation-rmse:6.56535
[4]	validation-rmse:6.54529
[5]	validation-rmse:6.53359
[6]	validation-rmse:6.52515
[7]	validation-rmse:6.51729
[8]	validation-rmse:6.50822
[9]	validation-rmse:6.50276
[10]	validation-rmse:6.49784
[11]	validation-rmse:6.49200
[12]	validation-rmse:6.48839
[13]	validation-rmse:6.48429
[14]	validation-rmse:6.47899
[15]	validation-rmse:6.47637
[16]	validation-rmse:6.47158
[17]	validation-rmse:6.46852
[18]	validation-rmse:6.46433
[19]	validation-rmse:6.45951
[20]	validation-rmse:6.45500
[21]	validation-rmse:6.45297
[22]	validation-rmse:6.45133
[23]	validation-rmse:6.44966
[24]	validation-rmse:6.44733
[25]	validation-rmse:6.44487
[26]	validation-rmse:6.44334
[27]	validation-rmse:6.43968
[28]	validation-rmse:6.43791
[29]	validation-rmse:6.43641
[30]	validation-rmse:6.43542
[31]	validation-rmse:6.43180
[32]	validation-rmse:6.42937
[33]	validation-rmse:6.42771
[34]	validation-rmse:6.4

2025/10/07 10:26:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/exp-tracking-env/lib/python3.10/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [10:26:45] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/07 10:26:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [78]:
import mlflow
print(mlflow.get_tracking_uri())
print(mlflow.get_experiment_by_name("nyc-taxi-experiment"))


sqlite:///mlflow.db
<Experiment: artifact_location='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1759671388532, experiment_id='2', last_update_time=1759671388532, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>


In [79]:
import mlflow
print("Active run:", mlflow.active_run())


Active run: <ActiveRun: >


In [80]:
# with mlflow.start_run():
    